# Objective 1: Image Classifier using CNN

|Name|Class|Admin Number|
|:----:|:----:|:----:|
|Muhammad Fitri Amir bin Abdullah|DAAA/FT/2B/06|2222811|

<br>

In this task, I aim to implement an image classifier using a Convolutional Neural Network in order to classify a dataset that contains 15 different types of vegetables. There are some steps given that I will need to adhere to such as: 
- converting the image into grayscale (1 channel instead of 3)
- making 1 model for 2 inputs, 31x31 pixels and 128x128 pixels, NOT 224.

## Importing Packages

## Exploratory Data Analysis

## Feature engineering

## Baseline Modelling

## Model Improvement